In [ ]:

# # **EDA датасета по сердечно-сосудистым заболеваниям**
# 
# ## **Введение**
# 
# EDA датасета по сердечно-сосудистым заболеваниям проводится с целью оценки структуры, целостности и полноты данных, а также обнаружения выбросов и аномалий.

# ## **Подготовка к работе**
# Импортируем необходимые для обработки данных библиотеки и загружаем данные

In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_id = "19T-Gm5Dovnae7htHjJP2gUt3Tyl6NPp2"  # ID файла на Google Drive
file_url = f"https://drive.google.com/uc?id={file_id}"

try:
    df = pd.read_csv(file_url, delimiter=';')
    print("Данные загружены")
except Exception as e:
    print("Произошла ошибка при загрузке файла:", e)
    raise   

print(f"Размер датасета: {df.shape}")
print(f"Количество строк: {df.shape[0]}")
print(f"Количество столбцов: {df.shape[1]}")
print("\n")

print("Выведем первые 5 строк датасета")
print(df.head())
print("\n")

print("Узнаем информацию о типах данных")
print(df.info())
print("\n")


# ## **Оценка полноты данных (completeness)**

In [ ]:


print('Полнота данных по колонкам')
# Создаем таблицу с результатами
results = []
for column in df.columns:
    total_rows = len(df)
    missing_count = df[column].isna().sum()
    filled_count = total_rows - missing_count
    completeness = (filled_count / total_rows) * 100

    results.append({
        'Колонка': column,
        'Заполнено (%)': round(completeness, 1),
        'Пропусков': missing_count,
        'Заполнено строк': filled_count
    })

# Создаем DataFrame и сортируем по полноте
completeness_df = pd.DataFrame(results)
completeness_df = completeness_df.sort_values('Заполнено (%)')

# Показываем результаты
print(completeness_df.to_string(index=False))
print()

# Статистика по качеству данных
perfect_columns = len(completeness_df[completeness_df['Заполнено (%)'] == 100])
good_columns = len(completeness_df[completeness_df['Заполнено (%)'] >= 90])
bad_columns = len(completeness_df[completeness_df['Заполнено (%)'] < 80])
empty_columns = len(completeness_df[completeness_df['Заполнено (%)'] == 0])

print(f"Колонок полностью заполнено (100%): {perfect_columns}")
print(f"Колонок хорошо заполнено (>=90%): {good_columns}")
print(f"Колонок плохо заполнено (<80%): {bad_columns}")
print(f"Колонок полностью пустых (0%): {empty_columns}")

# При необходимости показываем проблемные колонки
problem_columns = completeness_df[completeness_df['Заполнено (%)'] < 80]
if len(problem_columns) > 0:
    print(f"\n Проблемные колонки (заполнено меньше 80%):")
    for _, row in problem_columns.iterrows():
        print(f"   - {row['Колонка']}: {row['Заполнено (%)']}%")

# Визуализация
print("График")
plt.figure(figsize=(10, 8))

# График полноты по колонкам
plt.barh(completeness_df['Колонка'], completeness_df['Заполнено (%)'])
plt.xlabel('Процент заполненности')
plt.title('completeness относительно колонок')
plt.axvline(x=80, color='red', linestyle='--', label='Порог 80%')
plt.legend()

plt.tight_layout()
plt.show()

# Рекомендации
print("Рекомендации")
if bad_columns == 0:
    print("Отличное качество данных! Все колонки заполнены хорошо.")
elif bad_columns < 5:
    print("Хорошее качество данных. Несколько колонок требуют внимания.")
else:
    print("Есть проблемы с качеством данных. Много колонок с пропусками.")


# Таким образом, в представленном датасете хорошее качество данных, отличающихся высокой полнотой, за исключением колонки Alcohol Intake

# ## **Оценка уникальности данных**

In [ ]:


# Проверка уникальности строк
duplicate_count = df.duplicated().sum()
uniqueness_score = 1 - (duplicate_count / len(df))

print("Оценка уникальности строк")
print(f"Всего строк в датасете: {len(df)}")
print(f"Найдено дубликатов: {duplicate_count}")
print(f"Оценка уникальности: {uniqueness_score:.3f}")

# Проверка
if uniqueness_score == 1.0:
    print("Все хорошо: Дубликатов нет")
else:
    print("Проблема: Есть дубликаты")


# Все строки в датасете уникальны

# ## **Поиск выбросов методом IQR**

# Проводим статистический анализ числовых данных с целью выявления выбросов

In [ ]:


numerical_columns = df.select_dtypes(include=[np.number]).columns

outliers_summary = {}

for col in numerical_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outliers_count = len(outliers)
    outliers_percentage = (outliers_count / len(df)) * 100

    outliers_summary[col] = {
        'outliers_count': outliers_count,
        'outliers_percentage': outliers_percentage,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }

    print(f"\n--- {col} ---")
    print(f"Выбросов: {outliers_count} ({outliers_percentage:.2f}%)")
    print(f"Границы выбросов: [{lower_bound:.2f}, {upper_bound:.2f}]")

# Визуализация выбросов
if len(numerical_columns) > 0:
    print("Визуализация выбросов")

    # Ограничим количество графиков для читаемости
    cols_to_plot = numerical_columns[:6]  # Первые 6 числовых колонок

    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.ravel()

    for i, col in enumerate(cols_to_plot):
        if i < len(axes):
            df.boxplot(column=col, ax=axes[i])
            axes[i].set_title(f'Выбросы: {col}')

    plt.tight_layout()
    plt.show()


# Теперь проведем анализ только категориальных колонок

In [ ]:


categorical_cols = df.select_dtypes(include=['object', 'category']).columns
print(f"Категориальных колонок: {len(categorical_cols)}")

for col in categorical_cols:
    print(f"\n{col}:")
    print(f"   Всего уникальных значений: {df[col].nunique()}")
    print(f"   Пропущенные значения: {df[col].isna().sum()}")

    value_counts = df[col].value_counts()
    print("Возможные значения:")
    for i, (value, count) in enumerate(value_counts.head().items(), 1):
        print(f"   {i}. {value}: {count} записей")

    # Визуализация для колонок с малым числом уникальных значений
    if df[col].nunique() <= 10:
        plt.figure(figsize=(8, 4))
        value_counts.head(10).plot(kind='bar')
        plt.title(f'Распределение: {col}')
        plt.xticks(rotation=45)
        plt.show()

print(f"\nАнализ завершен! Обработано {len(categorical_cols)} категориальных колонок.")


# ## **Выводы**
# 
# ### Структура
# Датасет содержит 16 признаков и 1000 строк
# Типы данных корректны для всех колонок
# 
# ### Полнота данных
# Все данные в датасете уникальны, а также имеют хорошее качество, т.к. практически все колонки полноценны.
# Однако колонка Alcohol Intake заполнена только на 66.0%. Соответсвтенно, для адекватного анализа данных в дальнейшем, чтобы сохранить объем выборки, необходимо удалить данный столбец.
# 
# ### Выбросы
# В числовых данных выбросов не обнаружено
# Из категориальных данных проблемы выявлены только в Alcohol Intake